<h1 style="color:blue">Praktikum 9</h1>
<h3 style="color:blue">Tekstitöötluse koodi pakendamine märgendajate abil. <br> Osalausestamine</h3>

Käesolevas praktikumis vaatame, kuidas oma tekstitöötluse koodi "pakendada" EstNLTK märgendaja kujule. Lingvistilise analüüsi poole pealt uurime osalausete tuvastajat ja selle kasutusvõimalusi.

## Märgendajate loomine

Selleks, et tekstianalüüsi kood oleks paremini hallatav, taaskasutatav ja teistega jagatav, on mõistlik järgida koodi ülesehitamisel kindlaid printsiipe. 
Näiteks tekstianalüüsi sisend ja väljund peaksid olema selgelt defineeritud -- eriti sellisel juhul, kui analüüsil on sõltuvused ning enne analüüsi teostamist tuleb tekitada mitmed teised analüüsid. 
Samuti peaks olema võimalik analüüse ja vaheetappe salvestada ning taastada failist (või andmebaasist). 
Seetõttu pakubki EstNLTK välja programmeerimisliidese tekstitöötluse koodi "pakendamiseks" -- teksti analüüsi teostab märgendaja (`Tagger`) ning analüüsi tulemuseks on märgenduskiht (`Layer`). 
Enamus tekstianalüüsi komponente, millega me seni oleme kokku puutunud (nt teksti lausestaja ja sõnestaja, morfoloogiline analüsaator, nimeüksuste tuvastaja jne), ongi loodud kui `Tagger`-id, mis opereerivad `Layer` objektidega.

Tekstianalüüsi pakendamine `Tagger` ja `Layer` abil tagab, et:

* märgenduskihtide loomisel teostatakse eel- ja järelkontrolli ning tänu sellele järgivad andmed ühtset struktuuri;
* märgendajate konfiguratsiooni ja märgenduskihte saab visualiseerida Jupyter Notebook'is;
* märgenduskihte saab salvestada JSON kujul failidesse / lugeda failidest;
* märgenduskihte saab salvestada vabavaralisse [Postgresql andmebaasi](https://github.com/estnltk/estnltk/tree/main/tutorials/storage) / lugeda andmebaasist. See muutub eriti kasulikuks suurte tekstimassivide töötlemisel;


Selleks, et luua ise märgendajaid ja märgenduskihte, tuleb natukene kursis olla ka Pythoni objekt-orienteeritud poolega -- hea on omada algteadmisi klasside ja objektide loomise kohta, samuti võiks tunda põhimõisteid. 
Kui vajad selles osas teadmiste uuendamist, vaata enne edasilugemist üle "Lisamaterjal: Sissejuhatus objekt-orienteeritud programmeerimisse".
Samuti on soovitav kõrvale lahti võtta Moodle's olev "EstNLTK baastarkuste materjal", kus on detailsemalt käsitletud kihtide ja märgendajate loomist. 

---

Vaatame nüüd märgendaja loomist lähemalt. Olgu eesmärgiks luua märgendaja, mis tuvastab tekstis tsitaadid ehk jutumärkide vahelised tekstilõigud. 
Näidistekst märgendaja katsetamiseks:

In [1]:
text_str = '''
Mees helistab arvutiabi telefonile ja kurdab: "Mu arvutil läks pilt eest ära!"
"Vaadake, kas kõik juhtmed on sees!" ütleb spetsialist.
"Ei saa, pime on!" ütleb hädaline.
"Pange siis tuli põlema."
"Ei saa, elektrit ei ole!"
'''

#### Märgenduskihi tekitamine

Sisuliselt toimub märgendaja sees märgenduskihi ehk `Layer` objekti tekitamine. 
Kõigepealt vaatame, kuidas käib kihi tekitamine eraldiseisvalt (st ilma märgendajata).
Selleks võtame kasutusele regulaaravaldise, mis tuvastab tekstist jutumärkidega ümbritsetud lõike.
Leiame iga lõigu alguse, lõpu ning tekstisisu:

In [2]:
import re
quotations_regexp = re.compile('("[^"]+")')
for match_obj in list( quotations_regexp.finditer( text_str ) ):
    print( match_obj.start(1),   # lõigu algus
           match_obj.end(1),     # lõigu lõpp
           match_obj.group(1) )  # lõigu sisu (sõne)

47 79 "Mu arvutil läks pilt eest ära!"
80 116 "Vaadake, kas kõik juhtmed on sees!"
136 154 "Ei saa, pime on!"
171 196 "Pange siis tuli põlema."
197 223 "Ei saa, elektrit ei ole!"


Edasi loome näidisteksi põhjal uue `Text` objekti ning seejärel ka uue märgenduskihi ehk `Layer` objekti:

In [3]:
from estnltk import Text, Layer

In [4]:
text_obj = Text( text_str )
text_obj

text
"Mees helistab arvutiabi telefonile ja kurdab: ""Mu arvutil läks pilt eest ära!""""Vaadake, kas kõik juhtmed on sees!"" ütleb spetsialist.""Ei saa, pime on!"" ütleb hädaline.""Pange siis tuli põlema.""""Ei saa, elektrit ei ole!"""


In [5]:
layer = Layer(name='quotations', attributes=(), text_object=text_obj)
layer.meta['allikas'] = 'anekdoot'

Kihi nimeks sai `'quotations'` ja atribuute kihil esialgu ei ole. Küll aga andsime kihile edasi info selle kohta, milline on kihile vastav tekst (`text_object=text_obj`). NB! Kui `text_object` jääb määramata, siis saab kihile küll märgendusi lisada, aga pole võimalik kuvada selle tekstilist sisu (st seda, millised märgendused millist tekstifragmenti katavad).

Nagu `Text` objektile, nii on ka kihile võimalik kaasa anda meta-andmeid atribuudi `.meta` abil (mis on sisuliselt sõnastik).

Esialgu on loodud kiht tühi:

In [6]:
layer

Layer(name='quotations', attributes=(), spans=SL[])

Kasutame nüüd jälle [finditer](https://docs.python.org/3.8/library/re.html#re.finditer) meetodit, et leida tekstist kõik jutumärkide-vahelised lõigud, võtame välja lõikude asukohad (algus ja lõpp-indeksid) ning lisame kihile `add_annotation` meetodi abil. Tulemuseks ongi märgendustega kiht:

In [7]:
for match_obj in quotations_regexp.finditer( text_obj.text ):
    quotation_start = match_obj.start(1)
    quotation_end   = match_obj.end(1)
    layer.add_annotation( (quotation_start, quotation_end) )

In [8]:
layer

Layer(name='quotations', attributes=(), spans=SL[Span('"Mu arvutil läks pilt eest ära!"', [{}]),
Span('"Vaadake, kas kõik juhtmed on sees!"', [{}]),
Span('"Ei saa, pime on!"', [{}]),
Span('"Pange siis tuli põlema."', [{}]),
Span('"Ei saa, elektrit ei ole!"', [{}])])

Rohkem infot märgenduskihtide loomise kohta (kuidas luua eri tüüpi kihte: alamkihte, mitmeseid kihte ja ümbriskihte) leiad _estnltk baastarkuste materjalist_ .

#### Märgendaja loomine

Objekt-orienteeritud disainipõhimõtet järgides peaks ka `Layer` objekt loodama mingi klassi/objekti sees. EstNLTK-s ongi selleks puhuks märgendaja ehk `Tagger` klass. 

Märgendaja loomiseks tuleb tekitada `estnltk.taggers.Tagger`-i alamklass ning defineerida:
  * klassimuutuja `conf_param`, mis sisaldab märgendaja konfiguratsiooni;
  * konstruktor `__init__`, mille ülesanne on algseadistada märgendaja (ja vajadusel laadida sisse tööks vajalikud ressursid);
  * `_make_layer_template` meetod, mis tekitab ja tagastab tühja kihi ( nn _kihi malli_ ), kus atribuudid on määratud vastavalt märgendaja algseadistusele;
  * `_make_layer` meetod, mille ülesanne on tekitada ( `_make_layer_template` abil) uus märgeduskiht, täita see andmetega ning tagastada;

<div class="alert alert-block alert-warning">
<h4><i>Aga kuidas see kõik kokku käib? ( veel tehnilisem jutt )</i></h4>
<br>
Seni oleme teksti märgendamiseks kasutanud <code>tag</code> meetodit, mille rakendamisel toimub aga "taustal" veel 3 meetodi rakendamine. 
Nendeks meetoditeks on (väljakutsumise järjekorras): <code>make_layer</code> =><code>_make_layer</code> => <code>_make_layer_template</code>. 
Kõik need meetodid panustavad <code>Layer</code> objekti loomisesse.
    
<code>Tagger</code>-i <code>tag</code> meetodis kutsutakse kihi loomiseks välja meetod <code>make_layer</code> ning kui kihi loomine õnnestub, riputatakse see <code>Text</code> objekti külge. 
    
Ka meetod <code>make_layer</code> (NB! ilma alakriipsuta alguses) ei tegele ise kihi tekitamisega, vaid kontrollib enne kihi tekitamist, kas kõik kihi loomiseks vajalikud sisendkihid on olemas, seejärel kutsub kihi tekitamiseks välja <code>Tagger</code>-i meetodi <code>_make_layer</code> ning kõige lõpus kontrollib, kas loodud kiht on valiidse struktuuriga (nt sisaldab deklareeritud atribuute).

Meetod <code>_make_layer</code> kutsub omakorda välja <code>_make_layer_template</code>-i, et tekitada uus märgendaja algseadistusele vastav kiht, täidab kihi andmetega ning tagastab selle. 

Meetodid <code>tag</code> ja <code>make_layer</code> on kõigil märgendajatel ühesugused, neid muuta ega üle defineerida pole tarvis; küll aga tuleb uue <code>Tagger</code>-i loomisel ise implementeerida <code>_make_layer</code> ja <code>_make_layer_template</code>.
<br><br>
Aga miks on ikkagi kihi loomine jagatud laiali eri meetodite vahel?
<br><br>
<code>tag</code> ja <code>make_layer</code> eristamine on vajalik selleks, et oleks võimalik luua nii lõppkasutaja jaoks mugavaid <code>Text</code>-iga seotud kihte kui ka nn eraldatud kihte (ingl <i>detached layers</i>), mis pole <code>Text</code>-i alla riputatud. Eraldatud kihte kasutab Postgres andmebaasiliides (kihte saab <code>Text</code>-ist eraldi andmebaasi salvestada) ning neid saab kasutada ka rakendustes, kus kihi loomine on ainult ajutine vahe-etapp -- kui loodud kihilt on informatsioon kätte saadud, pole põhjust seda <code>Text</code>-i külge kinnitada, vaid võib selle ka ära visata (nii hoiab näiteks kokku mäluruumi).
<br>
<code>make_layer</code> ja <code>_make_layer</code> eristamine võimaldab aga eraldada rutiinsed eel- ja järeltegevused (vajalike sisendkihtide kontroll ja märgenduste valideerimine) otsesest kihi loomisest.
<br>
<code>_make_layer</code> ja <code>_make_layer_template</code> võimaldavad eristada kihi malli (milline on kihi struktuur?) ning konkreetsete andmetega täidetud kihti. 
Selline eristus on vajalik Postgres andmebaasiliideses: kõigepealt fikseeritakse "kihi malli" abil andmete formaat (luuakse uus tabel), misjärel on võimalik juba konkreetsete andmetega täidetud kihtide salvestamine, mis võib toimuda näiteks paralleeltöötluse võtteid kasutades (st mitu lõime/protsessi võivad parallelselt teostada märgenduskihtide loomist ja salvestamist tabelisse).
</div>

Vaatame nüüd samm-sammult tsitaatide märgendaja defineerimist.

**1)** Impordime vajalikud klassid ning loome `QuotationsTagger`-i, mis on `Tagger`-i alamklass (ülemklassi nimi läheb alamklassi nime järele sulgudesse):
```python
from estnltk import Text, Layer
from estnltk.taggers import Tagger

class QuotationsTagger(Tagger):
    """Tags quotations in Text.""" 
    conf_param = ['quotations_regexp']
    ...
```
Kui märgendaja vajab oma tööks mingeid ressursse, siis nende nimed (ehk: vastavate isendimuutujate nimed) tuleb panna klassimuutujasse `conf_param`. Käesoleval juhul paneme sinna `quotations_regexp`, kuna meil on plaanis hoida selle nimega muutujas tsitaate eraldavat regulaaravaldist.

**2)** Loome konstruktori, kus on deklareeritud sisendkihid ja väljundkihi nimi, samuti väljundkihi atribuudid ning tööks vajalikud ressursid:
```python
    ...
    def __init__(self, # Väljundkihi nimi muudetavaks:
                       output_layer='quotations'):
        # sisend-väljundkihid ja atribuudid
        self.input_layers = []
        self.output_layer = output_layer
        self.output_attributes = ['length']
        # tööks vajalikud ressursid
        self.quotations_regexp = re.compile('("[^"]+")')
    ...
```
Sisendkihte (`input_layers`) antud juhul pole: regulaaravaldist saame rakendada "toorel tekstil", muid kihte tarvis pole. 
Väljundkihi nimi on vaikimisi `'quotations'`, aga seda saab konstruktori parameetriga muuta. 
Kui kihi nimi on muudetav, on meil võimalik märgendaja arendust versioneerida, st võime tekitada kihid nimedega `'quotations_v1'`, `'quotations_v2'` jne ning võrrelda neid omavahel.
Isendimuutujas `output_attributes` on loetletud väljundkihi atribuudid; käesolevas näites lisame väljundkihile atribuudi `'length'`, kuhu hakkame salvestama tsitaatide (sõnede) pikkuseid. 
Ning lõpuks kompileerime tsitaate eraldava regulaaravaldise ja riputame isendimuutuja `self.quotations_regexp` alla.

**3)** Loome `_make_layer_template` meetodi, mille ülesandeks on luua nn kihi mall (ehk siis tühi märgenduskiht, mis vastab märgendaja algseadistusele):

```python
    def _make_layer_template(self):
        return Layer(name=self.output_layer, attributes=self.output_attributes)
```

Kihi loomisel kasutame muutujaid `self.output_layer` ja `self.output_attributes`, mis annavad märgendaja konfiguratsiooni. 
Nii on tagatud, et konfiguratsiooni muutes (nt `output_layer` väärtust muutes) vastaks loodav märgenduskiht konfiguratsioonile. 

**4)** Loome `_make_layer` meetodi, mille ülesandeks on luua etteantud `Text` objekti põhjal uus märgenduskiht:
```python
    ...
    def _make_layer(self, text, layers, status):
        # Loome uue kihi
        layer = self._make_layer_template()
        layer.text_object = text
        # Täidame kihi andmetega
        for match in self.quotations_regexp.finditer(text.text):
            quotation_start  = match.start(1)
            quotation_end    = match.end(1)
            annotation       = {'length':quotation_end - quotation_start}
            layer.add_annotation( (quotation_start, quotation_end), annotation )
        # Tagastame loodud kihi
        return layer
```
Meetod `_make_layer` saab argumentideks märgendatava `text`-i (`Text` objekti), olemasolevate kihtide sõnastiku `layers` (võtmeteks `Text` objekti kihinimed ja väärtusteks neile vastavad kihid) ning sõnastiku `status` (sinna võib salvestada metaandmeid kihi loomise protsessi kohta, _a la_ kas loomine õnnestus või kui palju see aega võttis).

Edasi luuakse `_make_layer_template` abil uus (tühi) kiht ning seotakse see ühepoolselt `Text` objektiga. 

Nagu eelnevas tsitaatide märgendamise näites, nii tuvastame ka nüüd võimalike tsitaatide asukohad [finditer](https://docs.python.org/3.8/library/re.html#re.finditer) abil.
Lisaks leiame iga tuvastatud tsitaadi pikkuse (sümbolites) ning paneme sõnastikku `annotation` võtme `'length'` alla.
Meetod `add_annotation` võtab 2 sisendargumenti: märgendatav tekstipositsioon `(quotation_start, quotation_end)` ning märgenduse sisu sõnastiku `annotation` kujul -- selle sõnastiku põhjal tekitatakse märgendusele atribuudid. 

Töö lõpus tagastame loodud kihi.

Meetodis `_make_layer`  meil olemasolevate kihtide sõnastikku `layers` vaja ei läinudki, kuna `self.input_layers = []` ehk märgendaja ei nõudnud sisendkihte. 
Aga kui sisendkihid oleks nõutud, tuleks need sõnastikust välja noppida (nt `input_layer_1 = layers[ self.input_layers[0] ]`, `input_layer_2 = layers[ self.input_layers[1] ]` jne) ning seejärel saaks neid kasutada uue kihi tekitamisel.
Oluline on rõhutada, et kõik nõutud kihid tuleb võtta just sõnastikust `layers`, mitte `text`-i alt (nagu seda tehakse tavaliselt). Miks? Märgendajat peab olema võimalik kasutada ka eraldatud kihtide loomiseks ehk siis selliste kihtide loomiseks, millest ükski ei ole veel `text`-i alla riputatud.

Paneme nüüd `QuotationsTagger`-i koodi kokku üheks klassiks:

In [9]:
import re

from estnltk import Text, Layer
from estnltk.taggers import Tagger

class QuotationsTagger(Tagger):
    """Tags quotations in Text.""" 
    conf_param = ['quotations_regexp']
    
    def __init__(self, # Väljundkihi nimi muudetavaks:
                       output_layer='quotations'):
        # sisend-väljundkihid ja atribuudid
        self.input_layers = []
        self.output_layer = output_layer
        self.output_attributes = ['length']
        # tööks vajalikud ressursid
        self.quotations_regexp = re.compile('("[^"]+")')

    def _make_layer_template(self):
        return Layer(name=self.output_layer, attributes=self.output_attributes)
        
    def _make_layer(self, text, layers, status):
        # Loome uue kihi
        layer = self._make_layer_template()
        layer.text_object = text
        # Täidame kihi andmetega
        for match in self.quotations_regexp.finditer(text.text):
            quotation_start  = match.start(1)
            quotation_end    = match.end(1)
            annotation       = {'length':quotation_end - quotation_start}
            layer.add_annotation( (quotation_start, quotation_end), annotation )
        # Tagastame loodud kihi
        return layer

Katsetame. Loome uue märgendaja:

In [10]:
quotes_tagger = QuotationsTagger()
quotes_tagger

QuotationsTagger(input_layers=(), output_layer=quotations, output_attributes=('length',), quotations_regexp=re.compile('("[^"]+")'))

Nüüd, kui märgendaja on olemas, saab seda teksti märgendamiseks välja kutsuda kahel viisil. 

**1)** Meetod `make_layer` tekitab uue kihi ja tagastab selle, aga ei riputa kihti `text`-i külge -- tegemist on nn eraldatud kihiga:

In [11]:
text = Text(text_str)
quotes_tagger.make_layer(text)

Layer(name='quotations', attributes=('length',), spans=SL[Span('"Mu arvutil läks pilt eest ära!"', [{'length': 32}]),
Span('"Vaadake, kas kõik juhtmed on sees!"', [{'length': 36}]),
Span('"Ei saa, pime on!"', [{'length': 18}]),
Span('"Pange siis tuli põlema."', [{'length': 25}]),
Span('"Ei saa, elektrit ei ole!"', [{'length': 26}])])

In [12]:
# Tekstil pole veel ühtegi kiht
text.layers

set()

**2)** Meetod `tag` tekitab uue kihi ning tagastab `Text` objekti-i, kuhu kiht on külge riputatud:

In [13]:
quotes_tagger.tag(text)

Text(text='\nMees helistab arvutiabi telefonile ja kurdab: "Mu arvutil läks pilt eest ära!"\n"Vaadake, kas kõik juhtmed on sees!" ütleb spetsialist.\n"Ei saa, pime on!" ütleb hädaline.\n"Pange siis tuli põlema."\n"Ei saa, elektrit ei ole!"\n')

In [14]:
# uurime külge riputatud kihti
text.quotations.display()

Mees helistab arvutiabi telefonile ja kurdab: "Mu arvutil läks pilt eest ära!" "Vaadake, kas kõik juhtmed on sees!" ütleb spetsialist. "Ei saa, pime on!" ütleb hädaline. "Pange siis tuli põlema." "Ei saa, elektrit ei ole!"

<div class="alert alert-block alert-warning">
<h4><i>Märgendajad vs ümbermärgendajad</i></h4>
<ul>
<li>Kui <code>Tagger</code> on mõeldud uue kihi tekitamiseks, siis lisaks on olemas veel  <code>Retagger</code>, mis tegeleb olemasoleva kihi muutmise või parandamisega. Näiteks on <code>Retagger</code>-id morfoloogilise analüüsi kihti muutvad <code>VabamorfDisambiguator</code> (lausepõhine ühestaja) ja <code>UserDictTagger</code> (kasutajasõnastikupõhine parandaja). <code>Retagger</code> liides kasutab samu atribuute, mis <code>Tagger</code>, aga meetodite nimed on teistsugused: <code>_make_layer(...)</code> asemel on <code>_change_layer(...)</code>, <code>make_layer(...)</code> asemel <code>change_layer(...)</code> ning <code>tag(...)</code> asemel <code>retag(...)</code>;</li>
<li>Detailsemat informatsiooni ja lisanäiteid märgendajate / ümbermärgendajate tekitamise kohta leiad estnltk baastarkuste materjalist.</li>
</ul>
</div>

### Ülesanne 1. Naise- ja mehenimede mainimiste analüüs (4 p)

Esimeseks ülesandeks on luua märgendaja (`Tagger` alamklass), mis määrab kõigi isikunimena märgendendatud nimeüksuste soo. 
Ehk siis: tuvastab, kas tegemist on naise- või mehenimega. 
Ülesande teises osas tuleb märgendaja abil analüüsida nimede/sugude mainimissagedusi ajakirjanduskorpuses.

#### 1.1 `NameGenderTagger` (2 p) 

Loo märgendaja (`Tagger`), mille sisendkihid on _nimeüksuste_ ja _morfoloogiliste analüüside kihid_, ning mis määrab kõigile isikunimena märgendendatud nimeüksustele soo.

Ette on antud funktsioon `name_gender_finder`, mis juba teostab nimede soo määramist, kasutades  sugude määramisel Vikipeedia nimeloendeid (pärinevad [siit](https://et.wikipedia.org/wiki/Naisenimede_loend) ja [siit](https://et.wikipedia.org/wiki/Mehenimede_loend)).

Eesmärgiks on kirjutada funktsioon ümber EstNLTK `Tagger`-i kujule: märgendaja `__init__` meetodis peab toimuma ressursside (nimeloendite) sisselaadimine ja märgendaja algseadistamine, `_make_layer_template` peab looma tühja algseadistatud kihi ning `_make_layer` peab täitma kihi märgendustega ning tagastama selle.

In [15]:
def name_gender_finder( text, male_names_file='mehenimed.txt', female_names_file='naisenimed.txt' ):
    # kontrollime, kas vajalikud sisendkihid on olemas
    assert 'ner' in text.layers, '(!) The input text is missing "ner" layer '
    assert 'morph_analysis' in text.layers, '(!) The input text is missing "morph_analysis" layer '
    # loeme sisse nimede loendid
    male_names = dict()
    with open(male_names_file, 'r', encoding='utf-8') as in_f:
        for line in in_f:
            line = line.strip()
            male_names[line] = 1
    female_names = dict()
    with open(female_names_file, 'r', encoding='utf-8') as in_f:
        for line in in_f:
            line = line.strip()
            female_names[line] = 1
    # Itereerime üle sisendkihtide (nimeüksused ja morf analüüsid)
    morph_word_index = 0
    morph_layer = text[ 'morph_analysis' ]
    for ner_phrase in text[ 'ner' ]:
        if ner_phrase.nertag == 'PER' or list(ner_phrase.nertag) == ['PER']:
            # Leiame morf analüüside kihi abil nimeüksuse lemmad
            phrase_start = ner_phrase.start
            phrase_end = ner_phrase.end
            phrase_lemmas = []
            while morph_word_index < len( morph_layer ):
                morph_word = morph_layer[ morph_word_index ]
                if phrase_start <= morph_word.start and \
                   morph_word.end <= phrase_end:
                    lemmas = list( set(morph_word.lemma) )
                    phrase_lemmas.extend( lemmas )
                    phrase_lemmas.append( '***' ) # märgime sõnalõpu
                elif phrase_end < morph_word.start:
                    break
                if morph_word.end == phrase_end:
                    break
                morph_word_index += 1
            # Otsustame lemmade põhjal nime soo (vaikimisi on sugu määramata)
            gender = 'UNK'
            for lemma in phrase_lemmas:
                if lemma in male_names:
                    gender = 'M'
                    if lemma in female_names:
                        # ambiguous gender: can be both m/f
                        gender = 'AMB'
                    break
                elif lemma in female_names:
                    gender = 'F'
                    break
                if lemma == '***':
                    # Vaatame ainult nime esimest sõna (ehk eesnime)
                    break
            print(' Leidsin nime: ', text.text[phrase_start:phrase_end], 
                           ' sugu:', gender)

In [16]:
# Näide funktsiooni rakendamisest
from estnltk import Text
from estnltk.taggers import NerTagger
ner_tagger = NerTagger()
# Loome näidisteksti
text = Text('''
Siim Pohlak palus riigikogu kõnepuldis elukaaslase kätt

Kalle Laanet (Reformierakond): kas ma olen õigesti aru saanud, et teie ei ole abielus ja kui te ei ole abielus, siis miks te ei ole?
Siim Pohlak: ma ei tea, kas te rahuldute selle vastusega, et jah, mul on pikalt olnud plaan oma laste ema omale abikaasaks kosida.

Oudekki Loone (Keskerakond): kui hakatakse korraldama rahvahääletust, hakatakse tegema kampaaniat. Kuidas referendumite rahastamise kampaaniatega on? Kas sihtasutuste toetused peaks olema valikud? 
Siim Pohlaku sõnul on seda arutamise koht. 

Kaja Kallas (Reformierakond): kas on võimalus, et koalitsiooni sõnastus muutub.
''').tag_layer()
# Märgendame nimeüksused
ner_tagger.tag(text)
# Määrame nimede soo
name_gender_finder( text )

 Leidsin nime:  Siim Pohlak  sugu: M
 Leidsin nime:  Kalle Laanet  sugu: M
 Leidsin nime:  Siim Pohlak  sugu: M
 Leidsin nime:  Oudekki Loone  sugu: F
 Leidsin nime:  Siim Pohlaku  sugu: M
 Leidsin nime:  Kaja Kallas  sugu: F


Lisaks nimede märgendamisele tuleks kihi ka loendada leitud nimesid ning kihi metaandmete alla salvestada loenduse tulemused. 
Seega, rakendatuna eelmisel näitetekstil peaks `NameGenderTagger` tekitama taolise kihi:

![alt text](valjundikihi_naide.png "Näide märgendatud kihist")

Maksimumpunktid märgendaja eest saab kui: 1) sisend- ja väljundkihid on korrektselt defineeritud ning kihinimed (konstruktori kaudu) muudetavad; 2) konstruktori kaudu saab määrata ressursifailide nimesid, 3) märgendaja tööks vajalike ressursside ettevalmistus toimub konstruktoris  ning `_make_layer` kasutab märgendamisel juba olemasolevaid ressursse (isendmuutujaid).

#### 1.2 Nimede mainimine sugude lõikes ajakirjanduskorpuses (2 p) 

Rakenda morfoloogilist analüüsi ja `NameGenderTagger`-it kataloogi `'aja_pm_EPL'` ajaleheartiklitel ning leia: 1) kõigi mainitud isikunimede jaotus sugude kaupa; 2) mainitud isikunimede jaotus sugude kaupa rubriikide lõikes. 

   * **NB!** Korpuses on juba märgendatud nimeüksused (vt kiht nimega `'named_entities_flat'`), seega nimeüksuste kihti uuesti looma ei pea. `NameGenderTagger`-i peaks olema võimalik nii rakendada, et `__init__` meetodi kaudu saab vajadusel ümber muuta nimeüksuste sisendkihi nime;
   * Igal artiklil on ka metaandmed; artikli rubriik on metaandmetes võtme `'rubriik'` all;

Arutelu (kirjalikud vastused):

   * Kui suur on _määramata_ sooga isikunimede osakaal? Kas see mõjutab loendustulemuste usaldusväärsust?
    
   * Uuri tulemusi detailsemalt ja paku vähemalt üks viis, kuidas saaks sugude määramise funktsionaalsust paremdada?

---

## Osalausestamine

Osalausestaja abil saab pikad ja keeruka struktuuriga laused jagada väiksemateks/lihtsamateks juppideks: osalauseteks. 
Enamasti ongi osalaused lihtlause-sarnased: sisaldavad ühte peaverbi (finiitverbi) ning sellega süntaktiliselt seotud sõnu. 

Vaatame üht näidet osalausestaja kasutamise kohta:

In [17]:
from estnltk import Text

text = 'Igaüks, kes traktori eest miljon krooni lauale laob, on huvitatud sellest, '+\
       'et traktor meenutaks lisavõimaluste poolest võimalikult palju kosmoselaeva.'

# Osalausestame teksti:
text = Text(text).tag_layer('clauses')

# Kõik osalaused
text.clauses

Layer(name='clauses', attributes=('clause_type',), spans=SL[EnvelopingSpan(['Igaüks', 'on', 'huvitatud', 'sellest', ','], [{'clause_type': 'regular'}]),
EnvelopingSpan([',', 'kes', 'traktori', 'eest', 'miljon', 'krooni', 'lauale', 'laob', ','], [{'clause_type': 'embedded'}]),
EnvelopingSpan(['et', 'traktor', 'meenutaks', 'lisavõimaluste', 'poolest', 'võimalikult', 'palju', 'kosmoselaeva', '.'], [{'clause_type': 'regular'}])])

 * _Tehniline märkus:_ nagu ajaväljendite tuvastaja ja süntaksianalüsaator (Maltparser), nii kasutab ka EstNLTK osalausestaja `java`-t. Seetõttu läheb esmakordsel `tag_layer(['clauses'])` väljakutsel tavaliselt natukene rohkem aega, kuna toimub `java` komponendi initsialiseerimine;

Osalausestaja eristab kahte liiki osalauseid:
   - `regular` -- tavalised osalaused, mis on tüüpiliselt üksteisest eraldatud koma ja/või sidesõnade (ja, ning, et, sest, kuid vms) abil; eelmises näites on tavalised osalaused "Igaüks on huvitatud sellest" ja "et traktor meenutaks lisavõimaluste poolest võimalikult palju kosmoselaeva." ;
   - `embedded` -- kiillaused, mis on sisestatud mingi lause või osalause sisse ning jagavad selle kaheks tükiks\*; eelmises näites: "kes traktori eest miljon krooni lauale laob" on kiillause lause "Igaüks on huvitatud sellest" sees; 
     
     \* NB! Osalausestaja kiillausete käsitlus ei ühti eesti keele traditsiooniliste grammatikakirjelduste EKG II ja [EKK](http://www.eki.ee/books/ekk09/index.php?id=482&p=5&p1=3) kiillausete definitsiooniga. Milliseid süntaktilisi üksusi ja miks osalausestajas kiillauseteks nimetatakse, seda selgitab detailsemalt programmi kirjeldav [Kaalepi ja Muischneki artikkel]( http://arhiiv.rakenduslingvistika.ee/ajakirjad/index.php/aastaraamat/article/view/ERYa8.04)

Milleks on osalausete tuvastajat tarvis? Osalausestatud tekstide peal on võimalik uurida mitmeid keelenähtuseid, mida automaatne  süntaksianalüüs veel lahendada ei oska või mille lahendus vajab täiustamist. 

Üheks selliseks keelenähtuseks on _kollokatsioonid_ ehk sõnade koosesinemised, millega sageli kaasneb ka eraldiseisev/metafoorne tähendus ( nt "hea seisma", "hambasse puhuma", "õppust võtma" ). Kollokatsioonide automaatne tuvastamine nõuab kõigepealt, et piiritleme konteksti, mille seest kollokatsioonikandidaate otsime. Kui kontekstiks valida terve lause, siis liitlausete puhul jääb sõelale üksjagu "prahti" ehk siis selliseid sõnapaare, mis ületavad osalausepiire ning ei saa olla kollokatsioonid. Seega saab osalausestamise tulemust aluseks võttes leida täpsemad kollokatsioonikandidaadid.

Teiseks keelenähtuseks on sõnadevahelised _rektsiooniseosed_ ehk süntaksiseosed, kus üks sõna määrab teise olemasolu ja vormi lauses. Näiteks, verb 'hoolima' tingib sageli seestütlevas käändes sõna esinemise lauses ( "Kas sa hoolid **temast** ?" ) ning verb 'tutvustama' osastavas ja alaleütlevas käändes sõnad ( "Konsul tutvustab **kuurorti** ka oma **kaasmaalastele**." ). Ka rektsiooniseoste uurimisel aitab osalausestamine täpsemalt piiritleda konteksti, mille sees seosed võivad sõnade vahel esineda (rektsiooniseosed tüüpiliselt ei ületa osalausepiire), ning seega parandada analüüsi kvaliteeti.

### Osalausestamine ja komavead

Osalausestaja eeldab vaikimisi, et ortograafiareegleid on järgitud täpselt ning komad paiknevad seal, kus nad peaksid olema. Seega, kui analüüsitavast tekstist on mõni osalauseid eraldav koma puudu, siis kannatab ka osalausestamise kvaliteet.

Siiski sisaldab osalausestaja ka eksperimentaalset režiimi, mille puhul üritatakse osalausepiire määrata ka siis, kui komad on puudu. 
Et seda režiimi kasutada, tuleb luua uus osalausestaja (`ClauseSegmenter`), millel on sisse lülitatud komavigade-suhtes-vähetundlikkuse lipp (`ignore_missing_commas=True`):

In [18]:
from estnltk import Text
from estnltk.taggers import ClauseSegmenter

# Tekitame komavigade-suhtes-vähetundliku osalausestaja
# Muudame väljundkihi nime nii, et see oleks eristatav tavalisest osalausete kihist
new_segmenter = ClauseSegmenter( output_layer='clauses_ignore_missing_commas', 
                                 ignore_missing_commas=True )

# Tekitame analüüsitava näidisteksti
text = Text('Keegi teine ka siin ju kirjutas et ütles et saab ise asjadele järgi minna '+\
            'aga vastust seepeale ei tulnudki.')
# Lisame nõutud sisendkihid
text.tag_layer(['words','sentences','morph_analysis'])

# Lisame osalausete märgenduse ja uurime tulemusi
new_segmenter.tag(text)
text['clauses_ignore_missing_commas']

Layer(name='clauses_ignore_missing_commas', attributes=('clause_type',), spans=SL[EnvelopingSpan(['Keegi', 'teine', 'ka', 'siin', 'ju', 'kirjutas'], [{'clause_type': 'regular'}]),
EnvelopingSpan(['et', 'ütles'], [{'clause_type': 'regular'}]),
EnvelopingSpan(['et', 'saab', 'ise', 'asjadele', 'järgi', 'minna'], [{'clause_type': 'regular'}]),
EnvelopingSpan(['aga', 'vastust', 'seepeale', 'ei', 'tulnudki', '.'], [{'clause_type': 'regular'}])])

In [19]:
# Võrdluseks uurime ka tavalist osalausete kihti
text.tag_layer(['clauses'])
text['clauses']

Layer(name='clauses', attributes=('clause_type',), spans=SL[EnvelopingSpan(['Keegi', 'teine', 'ka', 'siin', 'ju', 'kirjutas', 'et', 'ütles', 'et', 'saab', 'ise', 'asjadele', 'järgi', 'minna', 'aga', 'vastust', 'seepeale', 'ei', 'tulnudki', '.'], [{'clause_type': 'regular'}])])

Lisainformatsiooni osalausestaja kohta vt [EstNLTK dokumentatsioonist](https://github.com/estnltk/estnltk/blob/main/tutorials/nlp_pipeline/A_text_segmentation/06_clauses.ipynb).

<div class="alert alert-block alert-warning">
<h4><i>Lünkadega märgenduse eripärad</i></h4>
<br>
Osalausestuse puhul on tegemist nn lünkadega märgendusega -- kui ülem-osalause sees on kiillause, siis sisuliselt on seal keset osalauset lünk, mis jagab osalause kaheks tükiks. 
    
<b>Osalause tekstisisu.</b> Lünkadega märgenduse puhul tuleb arvestada sellega, et teksisisu tagastav <code>clause.enclosing_text</code> ei tööta korrektselt: kiillaused jäetakse ülem-osalausete seest välja võtmata. Seega <code>enclosing_text</code> asemel on parem kasutada:
<pre>
' '.join([word_span.text for word_span in clause])
</pre>
mis annab ligilähedaselt korrektse tulemuse -- eeldusel, et tühikute arv ja kuju osalauses pole olulised.
    
<b>Teksti tükeldamine.</b> Osalausete kihi järgi teksti tükeldamiseks saab kasutada funktsiooni <code>split_by()</code>:
<pre>
from estnltk_core.layer_operations import split_by
clauses = split_by(text, 'clauses')
</pre>
See toimib korrektselt eeldusel, et osalausete kihi nimetus on <code>'clauses'</code>. 
Kui kihi nime on muudetud (nt <code>'my_clauses'</code>), siis <code>split_by(text, 'my_clauses')</code> teostab küll tükeldamise, aga ei oska lünkadega arvestada: jätab kiillaused ülem-osalausete seest välja võtmata. 
Kui tahad kihi nimest sõltumatult saada korrektset osalauseteks tükeldust, siis selleks on funktsioon <code>split_by_clauses</code>:
<pre>
from estnltk_core.layer_operations import split_by_clauses
clauses = split_by_clauses(text, input_clauses_layer='my_clauses')
</pre>

<b>Itereerimine.</b> Selleks, et itereerida üle osalausete ja morf analüüside, võib kasutada:
<pre>
# pöördume otse osalausete kihi poole
for clause_spanlist in text.clauses:
    print(clause_spanlist.text)
    for word_span in clause_spanlist:
        # Kuna 'morph_analysis' on 'words' alamkiht, saame selle 
        # kaudu kätte morf analüüsid, sh sõnaliigid
        print('   ', word_span.text,'\t', word_span.morph_analysis.partofspeech)
</pre>
või
<pre>
# grupeerime morfoloogilised analüüsid (ehk siis: sõnad) osalausete kaupa
for clause_spanlist in text.morph_analysis.groupby( text.clauses ):
    print(clause_spanlist.text)
    for morph_span in clause_spanlist:
        print('   ',morph_span.text,'\t',morph_span.partofspeech)
</pre>
</div>

### Harjutus: osalausekihtide võrdlemine

Lisa lausele _"Minu arvates on see lahe ettevõtmine ja ma loodan väga et neil läheb hästi."_ tavaline osalausete kiht ning komavigade-suhtes-vähetundliku osalausestaja kiht.

Võrdle kahte kihti ning väljasta: 1) mõlemas kihis kattuvad osalaused, 2) osalaused, mille tuvastas ainult komavigade-suhtes-vähetundlik osalausestaja.

### Ülesanne 2. Võimalike komavigade tuvastamine (3 p)

Proovime nüüd kasutada osalausestajat komavigade tuvastamiseks. 

Ülesandeks on luua programm, mis märgendab tekstis puuduvaid komasid ehk toob esile sõnad, mille eest on koma puudu. 
Puuduvad komad saab kätte nii: tuleb luua tavaline osalausete kiht, komavigade-suhtes-vähetundlikku osalausestaja osalausete kiht ning võrrelda neid omavahel.
Kui komavigade-suhtes-vähetundlik osalausestaja tekitas osalause, mille alguspositsioon (`start`) ei lange kokku ühegi tavalise osalause alguspositsiooniga, siis on tõenäoliselt tegemist osalausega, mille ees peaks olema koma -- esile tulebki tuua sellise osalause esimene sõna.

Maksimumpunktid saab siis, kui komavigade tuvastaja on realiseeritud kui märgendaja (`Tagger` alamklass): märgendab tekstis sõnu, mille eest on koma puudu. 
Märgendaja sisendkihtideks (`input_layers`) peaks olema kõik need kihid, mis on `ClauseSegmenter`-i sisendkihtideks, pluss tavaline osalausete kiht (`'clauses'`).
Konstruktoris tuleb luua komavigade-suhtes-vähetundlik `ClauseSegmenter` ning salvestada isendimuutujaks.
Kihi tekitamine. Komavigade tuvastaja meetodis `_make_layer_template` luuakse kõigepealt uus tühi kiht, mis vastab konfiguratsioonile. 
Kihi andmetega täitmise loogika läheb aga komavigade tuvastaja meetodisse `_make_layer`.
Seal kasutatakse kõigepealt välja komavigade-suhtes-vähetundlik `ClauseSegmenter`, et tekitada sisendtekstile alternatiivne osalausete kiht. Kiht tuleks luua osalausestaja meetodi `make_layer` (NB! ilma alakriipsuta alguses) abil ehk siis tekkima peaks _eraldatud osalausete kiht_ .
Edasi tuleb võrrelda kaht osalausete kihti: tavalist `'clauses'` kihti ja komavigade-suhtes-vähetundliku osalausestaja kihti.
Kui komavigade-suhtes-vähetundlik osalausestaja tekitas osalause, mille alguspositsioon (`start`) ei lange kokku ühegi tavalise osalause alguspositsiooniga, siis on tõenäoliselt tegemist osalausega, mille ees peaks olema koma, ning märgendada tulebki sellise osalause esimene sõna.
Märgendaja testimise hõlbustamiseks on hea leitud komavigade arv salvestada märgenduskihi atribuudi `.meta` alla.

Testimine. Rakenda puuduvate komade tuvastajat failil `'foorumid_xml/Turg_valiidne.xml'` ning leia foorumiteema, mille postitustes oli kõige rohkem potentsiaalseid komavigasid.
Faili avamiseks ja üle teemade itereerimiseks saad kasutada [EstNLTK TEI korpuse sisselugemise vahendeid](https://github.com/estnltk/estnltk/blob/main/tutorials/corpus_processing/importing_text_objects_from_corpora.ipynb).
Täpsemalt siis funktsioone `get_div_target()` ja `parse_tei_corpus()`.
Tulemusena kuva kõige rohkem potentsiaalseid komavigu sisaldanud teema sisu, milles on ära märgitud puuduvate komadega sõnad (nt kasuta komavigade kihi `display()` meetodit).


---